# LRU Cache & Decorators – Practice Notebook

This notebook contains a series of **advanced (but not too advanced)** practice problems 
around **decorators**, **memoization**, and **LRU caches**.

Each exercise has:

- A description in a markdown cell
- A starter code cell (now fully implemented) 

You can run cells, experiment, and modify the implementations if you want to explore further. 
All TODOs have been implemented for you.


## Exercise 1 – Simple memoization decorator (positional args only)

Write a decorator `memoize_positional` that:

- Caches results **based only on positional arguments** (`*args`)
- Uses a simple dictionary as the cache
- Prints `"cache hit"` when a cached value is returned
- Does **not** support keyword-only arguments (it's OK to ignore `**kwargs` for this one)

Then:

1. Decorate the function `slow_add(a, b)` that:
   - Prints `f"computing slow_add({a}, {b})"` when called
   - Returns `a + b`
2. Call `slow_add(10, 20)` three times and verify:
   - The message is printed **only once**
   - Subsequent calls print `"cache hit"` instead


In [12]:
from functools import wraps

def memoize_positional(func):
    """Simple memoization decorator that only supports positional args.

    Caches results based on the tuple of positional arguments.
    """
    cache = {}

    @wraps(func)
    def inner(*args):
        key = args
        if key in cache:
            print("cache hit")
            return cache[key]
        result = func(*args)
        cache[key] = result
        return result

    return inner


@memoize_positional
def slow_add(a, b):
    print(f"computing slow_add({a}, {b})")
    return a + b


# Try these calls:
slow_add(10, 20)
slow_add(10, 20)
slow_add(10, 20)


computing slow_add(10, 20)
cache hit
cache hit


30

## Exercise 2 – Supporting both `*args` and `**kwargs`

Extend the idea from Exercise 1 to support **both positional and keyword arguments**.

Write a decorator `memoize` that:

- Accepts any combination of `*args` and `**kwargs`
- Builds a **hashable** cache key from:
  - The tuple of `args`
  - A `frozenset` of `kwargs.items()` (sorted or not – `frozenset` is fine)
- Prints:
  - `"cache miss"` when computing & storing a new value
  - `"cache hit"` when reusing a cached value

Then:

1. Decorate the function:

   ```python
   def power(base, exp=2):
       return base ** exp
   ```

2. Test that the following calls use the cache correctly:

   ```python
   power(2)
   power(2)
   power(2, exp=3)
   power(2, exp=3)
   power(base=2, exp=3)
   ```


In [13]:
def memoize(func):
    """Memoization decorator that caches based on both args and kwargs.

    The cache key is a tuple: (args, frozenset(kwargs.items())).
    """
    cache = {}

    @wraps(func)
    def inner(*args, **kwargs):
        key = (args, frozenset(kwargs.items()))
        if key in cache:
            print("cache hit")
            return cache[key]
        print("cache miss")
        result = func(*args, **kwargs)
        cache[key] = result
        return result

    return inner


@memoize
def power(base, exp=2):
    return base ** exp


# Try:
power(2)
power(2)
power(2, exp=3)
power(2, exp=3)
power(base=2, exp=3)


cache miss
cache hit
cache miss
cache hit
cache miss


8

## Exercise 3 – LRU cache with a fixed max size

Implement a **very small** LRU cache decorator `tiny_lru(maxsize)` using `collections.OrderedDict`.

Requirements:

- `tiny_lru` is a **decorator factory**:
  - Called as `@tiny_lru(maxsize=3)`
- Internally uses an `OrderedDict` that:
  - Stores keys in order of **recent use**
  - On each successful cache hit, moves the key to the *end* (most recently used)
  - When inserting a new key:
    - If the cache size would exceed `maxsize`, evicts the **least recently used** item
- For debugging, prints:
  - `"cache hit for {key}"`
  - `"cache miss for {key}"`
  - `"evicting {old_key}"` when an item is discarded

Then decorate:

```python
def mul(a, b):
    print(f"computing mul({a}, {b})")
    return a * b
```

with `@tiny_lru(maxsize=2)` and study the behavior for these calls:

```python
mul(1, 2)   # miss
mul(1, 2)   # hit
mul(2, 2)   # miss
mul(3, 3)   # miss, should evict the least recently used key
mul(1, 2)   # might be a miss again depending on eviction
```


In [14]:
from collections import OrderedDict

def tiny_lru(maxsize=128):
    """Very small LRU cache using OrderedDict.

    Only supports positional arguments for simplicity.
    """
    def decorator(func):
        cache = OrderedDict()

        @wraps(func)
        def inner(*args):
            key = args
            if key in cache:
                print(f"cache hit for {key}")
                # mark as recently used
                cache.move_to_end(key)
                return cache[key]

            print(f"cache miss for {key}")
            result = func(*args)
            cache[key] = result
            cache.move_to_end(key)

            if len(cache) > maxsize:
                old_key, _ = cache.popitem(last=False)
                print(f"evicting {old_key}")

            return result

        return inner

    return decorator


@tiny_lru(maxsize=2)
def mul(a, b):
    print(f"computing mul({a}, {b})")
    return a * b


# Try:
mul(1, 2)
mul(1, 2)
mul(2, 2)
mul(3, 3)
mul(1, 2)


cache miss for (1, 2)
computing mul(1, 2)
cache hit for (1, 2)
cache miss for (2, 2)
computing mul(2, 2)
cache miss for (3, 3)
computing mul(3, 3)
evicting (1, 2)
cache miss for (1, 2)
computing mul(1, 2)
evicting (2, 2)


2

## Exercise 4 – Combining logging and caching decorators

You are given two decorators:

- `log_calls` – logs function name and arguments
- `memoize` – from Exercise 2

### Part A

Implement `log_calls(func)` that:

- Prints a message **before** calling the function:

  ```text
  calling <name> with args=<args>, kwargs=<kwargs>
  ```

- Prints a message **after** the call:

  ```text
  <name> returned <result>
  ```

### Part B

Decorate the function `fib(n)` (naive recursive Fibonacci) in two different ways:

1. `@log_calls` then `@memoize` (i.e., `@memoize` is the *outer* decorator)
2. `@memoize` then `@log_calls`

And observe:

- How many log lines do you see for computing, say, `fib(10)`?
- What's the difference between the two decorator orders?

> 💡 This exercise is designed to help you understand how *decorator order* affects behavior.


In [ ]:
def log_calls(func):
    """Decorator that logs function calls and return values."""

    @wraps(func)
    def inner(*args, **kwargs):
        print(f"calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned {result}")
        return result

    return inner


# Option 1: memoize outside, log inside
# Uncomment to experiment after reading the output
#
@memoize
@log_calls
def fib_logged_then_cached(n):
    if n <= 1:
        return n
    return fib_logged_then_cached(n-1) + fib_logged_then_cached(n-2)


# Option 2: log outside, memoize inside
#
@log_calls
@memoize
def fib_cached_then_logged(n):
    if n <= 1:
        return n
    return fib_cached_then_logged(n-1) + fib_cached_then_logged(n-2)


# After uncommenting and defining, try:
fib_logged_then_cached(10)
fib_cached_then_logged(10)


## Exercise 5 – Measuring speedup with `functools.lru_cache`

In this exercise you'll:

1. Implement a **naive**, recursive Fibonacci function `fib_plain(n)` (no caching).
2. Implement a cached version `fib_cached(n)` using `@functools.lru_cache(maxsize=None)`.
3. Use `time.perf_counter()` to measure the time it takes to compute `fib_plain(35)` vs `fib_cached(35)`.

### Requirements

- Print the timings like:

  ```text
  fib_plain(35) took 0.8231 seconds, result=9227465
  fib_cached(35) took 0.00001 seconds, result=9227465
  ```

- Make sure the cached version calls `fib_cached(35)` **twice**:
  - First call warms up the cache
  - Second call should be almost instantaneous


In [16]:
from functools import lru_cache
from time import perf_counter

# naive recursive implementation (no caching)
def fib_plain(n):
    if n <= 1:
        return n
    return fib_plain(n - 1) + fib_plain(n - 2)


# cached implementation using lru_cache
@lru_cache(maxsize=None)
def fib_cached(n):
    if n <= 1:
        return n
    return fib_cached(n - 1) + fib_cached(n - 2)


def benchmark_fib():
    n = 35

    # plain version
    start = perf_counter()
    result_plain = fib_plain(n)
    end = perf_counter()
    print(f"fib_plain({n}) took {end - start:.6f} seconds, result={result_plain}")

    # cached version – first call (warms up cache)
    start = perf_counter()
    result_cached = fib_cached(n)
    end = perf_counter()
    print(f"fib_cached({n}) first call took {end - start:.6f} seconds, result={result_cached}")

    # cached version – second call (should be very fast)
    start = perf_counter()
    result_cached_2 = fib_cached(n)
    end = perf_counter()
    print(f"fib_cached({n}) second call took {end - start:.6f} seconds, result={result_cached_2}")


# Run this to compare performance:
benchmark_fib()


fib_plain(35) took 1.923673 seconds, result=9227465
fib_cached(35) first call took 0.000019 seconds, result=9227465
fib_cached(35) second call took 0.000000 seconds, result=9227465
